In [1]:
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.cross_validation import train_test_split

/usr/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
from pandas import DataFrame
from urllib2 import Request, urlopen
from json import dumps, loads

In [19]:
api = "http://api.parkwhiz.com/v4/quotes/?q=coordinates:41.8857256,-87.6369590&start_time=2017-12-23T12:00&end_time=2017-12-23T20:00&api_key=62d882d8cfe5680004fa849286b6ce20"

req = Request(api)
page = urlopen(req)
content = page.read()

json_data = loads(content)
json_data

51

In [18]:
columns = ['id','name','lat','lon','coordinates','address','city','seller_id','price', 'accessible', 'valet', 'security', 'restrooms', 'printed_pass', 'reentry_allowed']
frame = DataFrame(columns=columns)

for i, event in enumerate(json_data):
    try:
        dict0={}
        dict0['id'] = event['_embedded']['pw:location']['id']
        dict0['name'] = event['_embedded']['pw:location']['name']
        dict0['lat'] = float(event['_embedded']['pw:location']['entrances'][0]['coordinates'][0])
        dict0['lon'] = float(event['_embedded']['pw:location']['entrances'][0]['coordinates'][1])
        dict0['coordinates'] = event['_embedded']['pw:location']['entrances'][0]['coordinates']
        dict0['address'] = event['_embedded']['pw:location']['address1']
        dict0['city'] = event['_embedded']['pw:location']['city']
        dict0['seller_id'] = event['seller_id']
        dict0['price'] = float(event['purchase_options'][0]['price']['USD'])
        
        amenities = event['purchase_options'][0]['amenities']
        for item in amenities:
            if item['name'] == 'Accessible':
                dict0['accessible'] = item['enabled']
            if item['name'] == 'Valet':
                dict0['valet'] = item['enabled']
            if item['name'] == 'Security':
                dict0['security'] = item['enabled']
            if item['name'] == 'Restrooms':
                dict0['restrooms'] = item['enabled']
            if item['name'] == 'Printed Pass':
                dict0['printed_pass'] = item['enabled']
            if item['name'] == 'Reentry Allowed':
                dict0['reentry_allowed'] = item['enabled']
        # Checks if it is accessible, if there is valet, security, restrooms, printed pass, re-entry allowed
        for key in dict0:
            frame.loc[i,key] = dict0[key]
    except Exception:
        pass
frame.head()

,id,name,lat,lon,coordinates,address,city,seller_id,price,accessible,valet,security,restrooms,printed_pass,reentry_allowed
0,6460,Franklin & Lake Self Park,41.8852,-87.6356,"[41.885230118, -87.6355737448]",180 N. Franklin St.,Chicago,652,13,True,False,True,False,False,False
1,3871,225 W. Wacker Dr. Building,41.8859,-87.6353,"[41.8859381122, -87.6352616959]",281 N. Franklin St.,Chicago,602,9,True,False,False,False,False,False
2,9529,319 W. Randolph - Lot,41.8843,-87.636,"[41.8842716176, -87.6360226795]",319 W. Randolph St.,Chicago,652,13,True,False,False,False,False,False
3,6168,Wolf Point Self Park,41.8879,-87.6373,"[41.8879379045, -87.6372899022]",343 W. Wolfpoint Plz.,Chicago,652,17,False,False,False,False,False,False
4,3143,Self Park,41.8862,-87.634,"[41.8862067926, -87.6339764484]",210 N. Wells St.,Chicago,624,10,True,False,False,False,False,False


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(frame[['lat','lon']],frame['price'],test_size=0.2,random_state=574)

In [ ]:
knnR = KNeighborsRegressor(n_neighbors=3,metric='haversine')

In [ ]:
knnR.fit(X_train,y_train)

In [ ]:
from repository.mlrepositoryclient import MLRepositoryClient
from repository.mlrepositoryartifact import MLRepositoryArtifact
from repository.mlrepository import MetaProps, MetaNames

In [ ]:
wml_credentials = {
  "url": "https://ibm-watson-ml.mybluemix.net",
  "access_key": "5eJV3UHu5LQEVQObnm8ciAywGzvSm7y47quRrlI7Luq3lOzm9fi0i9yQkv70gO54HxGxQ3pIogjgEOjN0TGDTcL0h32gVzPkwMbmHXNpi+FQYUqQmv73SQJrb1WXWeZv",
  "username": "33e97d56-995f-4f77-b0f1-646731e50702",
  "password": "3c2a4a2d-7020-46c5-a9c8-954122ac9d60",
  "instance_id": "00195eb4-c665-4384-9e21-57e59ac66f5f"
}

In [ ]:
ml_repository_client = MLRepositoryClient(wml_credentials['url'])
ml_repository_client.authorize(wml_credentials['username'], wml_credentials['password'])

In [ ]:
props = MetaProps({MetaNames.AUTHOR_NAME:"Aleksandar Velkoski", MetaNames.AUTHOR_EMAIL:"avelkoski@ethventures.io"})
model_artifact = MLRepositoryArtifact(knnR, name="SpotExchange KNeighborsRegressor", meta_props=props)

In [ ]:
saved_model = ml_repository_client.models.save(model_artifact)

In [ ]:
import urllib3, requests, json

headers = urllib3.util.make_headers(basic_auth='{username}:{password}'.format(username=wml_credentials['username'], password=wml_credentials['password']))
url = '{}/v3/identity/token'.format(wml_credentials['url'])
response = requests.get(url, headers=headers)
mltoken = json.loads(response.text).get('token')

In [ ]:
endpoint_instance = wml_credentials['url'] + "/v3/wml_instances/" + wml_credentials['instance_id']
header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken} 

response_get_instance = requests.get(endpoint_instance, headers=header)

In [ ]:
endpoint_published_models = json.loads(response_get_instance.text).get('entity').get('published_models').get('url')

In [ ]:
print endpoint_published_models

In [ ]:
header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}
response_get = requests.get(endpoint_published_models, headers=header)

In [ ]:
print mltoken

In [ ]:
[endpoint_deployments] = [x.get('entity').get('deployments').get('url') for x in json.loads(response_get.text).get('resources') if x.get('metadata').get('guid') == saved_model.uid]

In [ ]:
payload_online = {"name": "knnR-SpotExchange", "description": "KNearestNeighbor Regressor for SpotExchange Price Predictions", "type": "online"}
response_online = requests.post(endpoint_deployments, json=payload_online, headers=header)

In [ ]:
scoring_url = json.loads(response_online.text).get('entity').get('scoring_url')

print(scoring_url)

In [ ]:
values = [[41.88523011803571, -87.63557374477386],
       [41.88593811224804, -87.6352616958502],
       [41.88427161764708, -87.63602267950773]]

In [ ]:
payload_scoring = {"values": values}

In [ ]:
response_scoring = requests.post(scoring_url, json=payload_scoring, headers=header)

In [ ]:
print response_scoring.text